# Rendering an Animated GIF
- - -
This notebook is a bare bones example of how to render an animated GIF using 
IPYCanvas. It leverages _moviepy_ of the video encoding.

## Known Limitations
At the moment this code doesn't work in VSCode. It's confirmed to work in 
Jupyter Notebook.

## Running the Notebook
To run the notebook.
1. Launch a poetry shell if one doesn't exist already.
   ```shell
   poetry shell
   ```
2. Start Jupyter Notebooks
   ```shell
   jupyter notebook
   # A link should be displayed to open the notebook.
   ```
3. Open the Notebook in a browser. Do this by opening the linking in the last step.
4. In the browser run the notebook. An the notebook should draw an animation on 
   the screen and save it as _test.gif_.

In [10]:
# Define the Renderer and Processing Thread
from ipywidgets import Output
from ipycanvas import Canvas, hold_canvas
from threading import Thread
import numpy as np
import time
from IPython.display import display, Image
from moviepy.editor import ImageSequenceClip

out = Output()

# 1000/125 = 8 FPS
TARGET_FPS = 8
SLEEP_TIME_SEC:float = 0.125

class Renderer:
  def __init__(self, canvas, save_image, ):
    self.frames = []
    self.canvas = canvas
    self.canvas.observe(self.get_array, 'image_data')
    self.setup_canvas()
    self.save_image = save_image
    
  @out.capture()
  def get_array(self, *args, **kwargs):
    if self.save_image:
      arr = self.canvas.get_image_data()
      self.frames.append(arr)
    
  def setup_canvas(self):
    self.canvas.fill_style = 'white'
    self.canvas.fill_rect(0,0,self.canvas.width, self.canvas.height)
    
  def render(self, x):
    self.canvas.stroke_line(50, 10+(x*5), 150, 10+(x*5))
      
class RenderingThread(Thread):
  def __init__(self, renderer, image_name):
    super(RenderingThread, self).__init__()
    self.renderer = renderer
    self.image_name = image_name

  def run(self):
    for i in range(8):
      self.renderer.render(i)
      time.sleep(SLEEP_TIME_SEC)
    self.on_complete()    

  def on_complete(self):
    if self.renderer.save_image:
      self.save_gif()
        
  @out.capture()
  def save_gif(self):
    print(f"Number of frames: {len(self.renderer.frames)}")
    clip = ImageSequenceClip(list(self.renderer.frames), fps=TARGET_FPS)
    clip.write_gif(self.image_name, fps=TARGET_FPS)
    # Image(self.image_name)

In [11]:
# The Main Entry Point

# Set this to False to do development in VSCode. 
# Set to True when running in Jupyter Notebooks
SAVE_ANIMATED_GIF = False

display(out)
canvas = Canvas(width=200, height=200, sync_image_data=True)
renderer = Renderer(canvas, SAVE_ANIMATED_GIF)
thread = RenderingThread(renderer, 'test.gif')

display(canvas)
thread.start()


Output()

Canvas(height=200, sync_image_data=True, width=200)